In [1]:
!pip install pandas
!pip install requests

import pandas as pd
import unicodedata
import datetime

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
url_componentes = 'componentes-graduacao.csv'
url_unidades = 'http://dados.ufrn.br/dataset/da6451a5-1a59-4630-bdc2-97f6be4a59c2/resource/3f2e4e32-ef1a-4396-8037-cbc22a89d97f/download/unidades.csv'
url_docentes = 'http://dados.ufrn.br/dataset/8bf1a468-48ff-4f4d-95ee-b17b7a3a5592/resource/ff0a457e-76fa-4aca-ad99-48aebd7db070/download/docentes.csv'
url_avaliacoes = 'http://dados.ufrn.br/dataset/d5723d75-7e6e-4264-82aa-b96909b69f63/resource/7accd1d2-2793-460e-b98d-87a0679b9155/download/avaliacaodocencia.csv'

In [3]:
df_unidades = pd.read_csv(url_unidades, ';')
df_unidades = df_unidades[['id_unidade_responsavel', 'unidade_responsavel']]
df_unidades.rename(columns={'id_unidade_responsavel': 'unidade', 'unidade_responsavel': 'lotacao'}, inplace=True)
df_unidades = df_unidades[(df_unidades['lotacao'] == 'DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA') | (df_unidades['lotacao'] == 'INSTITUTO METROPOLE DIGITAL')].drop_duplicates()
df_unidades.reset_index(inplace=True, drop=True)
df_unidades.head()

,unidade,lotacao
0,125,DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA
1,6069,INSTITUTO METROPOLE DIGITAL


In [4]:
df_docentes = pd.read_csv(url_docentes, ';')
df_docentes.columns

Index([u'id_servidor', u'siape', u'nome', u'formacao',
       u'tipo_jornada_trabalho', u'vinculo', u'categoria', u'classe_funcional',
       u'id_unidade_lotacao', u'lotacao', u'admissao', u'Unnamed: 11'],
      dtype='object')

In [5]:
df_docentes = df_docentes[df_docentes['categoria'] == 'PROFESSOR DO MAGISTERIO SUPERIOR']

In [6]:
df_docentes.rename(columns={'id_servidor': 'docente', 'admissao': 'data_admissao', 'id_unidade_lotacao': 'id_unidade'}, inplace=True)
df_docentes = df_docentes.merge(df_unidades[['unidade']], right_on='unidade', left_on='id_unidade', how='inner')
df_docentes['data_admissao'] = [ data.date() for data in pd.to_datetime(df_docentes['data_admissao'])]
df_docentes = df_docentes[['docente', 'nome', 'formacao', 'id_unidade', 'data_admissao']]
df_docentes.head()

,docente,nome,formacao,id_unidade,data_admissao
0,5755281,ADJA FERREIRA DE ANDRADE,DOUTORADO,6069,2012-11-01
1,5755819,ANDERSON PAIVA CRUZ,DOUTORADO,6069,2013-04-25
2,5757969,ANDRE LUIZ DA SILVA SOLINO,MESTRADO,6069,2016-01-19
3,5759298,ANNA GISELLE CAMARA DANTAS RIBEIRO RODRIGUES,DOUTORADO,6069,2017-05-26
4,3032089,APUENA VIEIRA GOMES,DOUTORADO,6069,2006-08-25


In [7]:
df_componentes = pd.read_csv(url_componentes, sep=',')
df_componentes = df_componentes[(df_componentes['unidade_responsavel'] == 'DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA') | (df_componentes['unidade_responsavel'] == 'INSTITUTO METROPOLE DIGITAL')]
df_componentes.rename(columns={'id_componente': 'componente'}, inplace=True)
df_componentes.head()

,componente,codigo,nivel,nome,unidade_responsavel
6,20007,DIM0072,G,TECNICAS DE ESPECIFICACAO FORMAL,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1033,21073,DIM0056,G,SOFTWARE BASICO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1043,21083,DIM0346,G,GERENCIAMENTO E SEGURANCA EM REDES DE COMPUTAD...,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1619,21667,DIM0046,G,ALGORITMOS E TECNICAS DE PROGRAMACAO,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA
1620,21668,DIM0003,G,ORGANIZACAO DE COMPUTADORES,DEPARTAMENTO DE INFORMÁTICA E MATEMÁTICA APLICADA


In [8]:
unidades_normalizadas = [unicodedata.normalize('NFKD', unicode(nome, "utf-8")).encode('ASCII', 'ignore') for nome in df_componentes['unidade_responsavel']]
series = pd.Series(data=unidades_normalizadas, index=df_componentes.index)
df_componentes['unidade_responsavel'] = series

In [9]:
df_componentes = df_componentes.merge(df_unidades[['unidade', 'lotacao']], left_on='unidade_responsavel', right_on='lotacao', how='inner')

In [10]:
df_componente_curricular = df_componentes[['componente', 'codigo']]
df_componente_curricular.head()

,componente,codigo
0,20007,DIM0072
1,21073,DIM0056
2,21083,DIM0346
3,21667,DIM0046
4,21668,DIM0003


In [11]:
df_componente_lotacao = df_componentes[['codigo', 'unidade', 'nome']]
df_componente_lotacao.head()

,codigo,unidade,nome
0,DIM0072,125,TECNICAS DE ESPECIFICACAO FORMAL
1,DIM0056,125,SOFTWARE BASICO
2,DIM0346,125,GERENCIAMENTO E SEGURANCA EM REDES DE COMPUTAD...
3,DIM0046,125,ALGORITMOS E TECNICAS DE PROGRAMACAO
4,DIM0003,125,ORGANIZACAO DE COMPUTADORES


In [12]:
turma_periodo = ['http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e77d066-d506-45eb-a21e-76aa79616fef/download/turmas-2017.1',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/01fe7343-fdf0-4a67-b915-2386b7c2fecb/download/turmas-2017.2',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/322d9977-ba15-47f1-8216-75a1ca78e197/download/turmas-2016.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e8e3228-7f22-40a2-9efd-561c44844567/download/turmas-2016.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/4d5aee5a-00b0-4ed6-a4be-59fa77a56797/download/turmas-2015.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/7c59621c-4a8b-49d4-b319-83cfea9bdf28/download/turmas-2015.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/e6e4144f-4042-4fdc-84e0-76e9ec27ae7c/download/turmas-2014.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/2c69547b-920f-4ec2-92c0-3fbc19512165/download/turmas-2014.2.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/0d96d930-8058-4def-9044-c3ae04c1f40c/download/turmas-2013.1.csv',
'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/e7e3cf12-a29b-491a-a895-021a43819197/download/turmas-2013.2.csv',]

In [13]:
frames = [pd.read_csv(periodo, sep=';', usecols=['id_turma', 'ano', 'nivel_ensino', 'periodo', 'id_componente_curricular']) for periodo in turma_periodo]

In [14]:
df_turmas = pd.concat(frames)

In [15]:
df_turmas.rename(columns={'id_turma': 'turma', 'nivel_ensino': 'nivel', 'id_componente_curricular': 'id_componente'}, inplace=True)
df_turmas = df_turmas[df_turmas['nivel'] == 'GRADUAÇÃO']
df_turmas.drop_duplicates('turma', inplace=True)
df_turmas['periodo'] = df_turmas['periodo'].astype(int) 
df_turmas['id_componente'] = df_turmas['id_componente'].astype(int)
df_turmas = df_turmas.merge(df_componente_curricular[['componente']], right_on='componente', left_on='id_componente', how='inner')
df_turmas = df_turmas[['turma', 'id_componente', 'nivel', 'ano', 'periodo']]
df_turmas.head()

,turma,id_componente,nivel,ano,periodo
0,57596647,2022854,GRADUAÇÃO,2017,1
1,57606915,2022854,GRADUAÇÃO,2017,2
2,57586681,2022854,GRADUAÇÃO,2016,2
3,1223534,2022854,GRADUAÇÃO,2014,1
4,1233980,2022854,GRADUAÇÃO,2014,2


In [16]:
df_avaliacoes = pd.read_csv(url_avaliacoes, sep=';', usecols=['id_docente', 'id_turma', 'qtd_discentes', 'postura_profissional_media', 'postura_profissional_DP', 'atuacao_profissional_media', 'atuacao_profissional_DP'])
df_avaliacoes = df_avaliacoes.merge(df_turmas[['turma']].astype(int), right_on='turma', left_on='id_turma', how='inner')
df_avaliacoes = df_avaliacoes.merge(df_docentes[['docente']].astype(int), left_on='id_docente', right_on='docente', how='inner')
df_avaliacoes.rename(columns={'postura_profissional_media': 'postura_profissional', 'atuacao_profissional_media': 'atuacao_profissional'}, inplace=True)
df_avaliacoes = df_avaliacoes[['id_docente', 'id_turma', 'qtd_discentes', 'postura_profissional', 'postura_profissional_DP', 'atuacao_profissional', 'atuacao_profissional_DP']]
df_avaliacoes.head()

,id_docente,id_turma,qtd_discentes,postura_profissional,postura_profissional_DP,atuacao_profissional,atuacao_profissional_DP
0,22764,1233659,17,9.09,1.57,9.00,1.60
1,22764,1233661,36,9.48,1.26,9.43,1.20
2,22764,1242644,34,9.41,1.04,8.86,1.73
3,22764,57566712,36,9.81,0.51,9.48,1.35
4,22764,57566818,14,9.69,0.48,8.13,2.44


In [17]:
matriculas = [
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/55dfe713-ff7c-4fa8-8d1d-d4294a025bff/download/matricula-componente-20172',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/79071c21-e32c-438f-b930-d1b6ccc02ec2/download/matricula-componente-20171',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/f6179838-b619-4d7d-af9c-18c438b80dd4/download/matriculas-de-2016.2.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/4778d3ce-8898-46a8-a623-ee6a480a2980/download/matriculas-de-2016.1.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/baa6c8b4-2072-417f-b238-c028ccc8c14b/download/matriculas-de-2015.2.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/9e7ba1c2-f92d-4b9c-9e91-3b026ecdf913/download/matriculas-de-2015.1.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/e974792c-b557-470c-bf3d-ede7d5b5e6a6/download/matricula-componente-20142.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/7081446d-39f9-4374-ad0b-86ecab97e569/download/matricula-componente-20141.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/517ed5f6-f8a2-40fd-826b-6ed3388f6e88/download/matricula-componente-20132.csv',
    'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/674de4cc-1fc0-4314-9f04-a38f0e1f9225/download/matricula-componente-20131.csv',
]

In [18]:
dfs_matricula = [pd.read_csv(matricula, sep=';') for matricula in matriculas]

In [19]:
df_matriculas = pd.concat(dfs_matricula)

In [20]:
df_matriculas = df_matriculas[~df_matriculas['descricao'].isin(['CUMPRIU', 'INDEFERIDO', 'EXCLUIDA'])]

In [21]:
df_matriculas.replace('APROVADO POR NOTA', 'APROVADO', inplace=True)
df_matriculas.replace(['REPROVADO POR NOTA', 'REPROVADO POR FALTAS', 'REPROVADO POR MÉDIA E POR FALTAS', 'REPROVADO POR NOTA E FALTA', 'DESISTENCIA', 'TRANCADO', 'CANCELADO'], 'REPROVADO', inplace=True)

In [22]:
df_matriculas = df_matriculas.merge(df_turmas[['turma']].astype(int), right_on='turma', left_on='id_turma', how='inner')
df_matriculas.head()

,Unnamed: 10,descricao,discente,faltas_unidade,id_curso,id_turma,media_final,nota,numero_total_faltas,reposicao,unidade,turma
0,NaN,APROVADO,bc87085ad93b23777125e06426688c0b,0.0,2000006.0,57606947,8.4,8.8,0.0,f,2.0,57606947
1,NaN,APROVADO,bc87085ad93b23777125e06426688c0b,0.0,2000006.0,57606947,8.4,8.3,0.0,f,1.0,57606947
2,NaN,APROVADO,bc87085ad93b23777125e06426688c0b,0.0,2000006.0,57606947,8.4,8.0,0.0,f,3.0,57606947
3,NaN,APROVADO,853a80ecb864882511c390f74aebbdfb,0.0,2000069.0,57606947,7.0,6.2,8.0,f,3.0,57606947
4,NaN,APROVADO,853a80ecb864882511c390f74aebbdfb,0.0,2000069.0,57606947,7.0,7.5,8.0,f,2.0,57606947


In [23]:
df_aprovados = df_matriculas[df_matriculas['descricao'] == 'APROVADO']
df_media_aprovados = df_aprovados.groupby('id_turma')['media_final'].mean().to_frame('media_aprovados').reset_index(level=0)

df_media_aprovados.head()

,id_turma,media_aprovados
0,1197615,7.010204
1,1197616,7.031250
2,1198464,6.727273
3,1198466,7.208696
4,1198499,6.451852


In [24]:
#media_turma = df_matriculas.groupby('id_turma')['media_final'].mean().to_frame('média').reset_index(level=0)
#media_turma.head()

In [25]:
df_porcentagem = df_matriculas.groupby('id_turma')['descricao'].value_counts(normalize=True).to_frame('aprovados')

In [26]:
df_porcentagem = df_porcentagem.reset_index(level=[0, 1])
df_porcentagem = df_porcentagem[~df_porcentagem['descricao'].isin(['REPROVADO'])]
df_porcentagem = df_porcentagem[['id_turma', 'aprovados']]
df_porcentagem.head()

,id_turma,aprovados
0,1197615,0.840000
2,1197616,0.578313
4,1198464,0.846154
7,1198466,0.482517
9,1198499,0.462857


In [27]:
df_avaliacoes = df_avaliacoes.merge(df_media_aprovados, on='id_turma', how='inner')
df_avaliacoes = df_avaliacoes.merge(df_porcentagem, on='id_turma', how='inner')
df_avaliacoes.head()

,id_docente,id_turma,qtd_discentes,postura_profissional,postura_profissional_DP,atuacao_profissional,atuacao_profissional_DP,media_aprovados,aprovados
0,22764,1233659,17,9.09,1.57,9.00,1.60,6.670588,0.739130
1,22764,1233661,36,9.48,1.26,9.43,1.20,6.842857,0.882353
2,22764,1242644,34,9.41,1.04,8.86,1.73,7.603030,0.916667
3,22764,57566712,36,9.81,0.51,9.48,1.35,7.578788,0.860870
4,22764,57566818,14,9.69,0.48,8.13,2.44,7.458333,0.837209


In [28]:
df_avaliacoes.to_csv('avaliacao.csv', sep=';', index=False)
df_componente_lotacao.to_csv('componente_lotacao.csv', sep=';', index=False)
df_componente_curricular.to_csv('componente_curricular.csv', sep=';', index=False)
df_docentes.to_csv('docente.csv', sep=';', index=False)
df_unidades.to_csv('unidade.csv', sep=';', index=False)
df_turmas.to_csv('turma.csv', sep=';', index=False)

In [35]:
import pandas as pd
import requests
import json

api_url = 'http://localhost:8080/'

def make_post(df, endpoint):
    records = json.loads(df.to_json(orient='records'))
    for record in records:
        request = requests.post(endpoint, data=json.dumps(record), headers={'Content-Type': 'application/json'})
        print request.text
        if not request.ok:
            break

def insert(df_url, endpoint):
    df = pd.read_csv(df_url, sep=';')
    make_post(df, endpoint)

def insert_componentes():
    df_lotacao = pd.read_csv('componente_lotacao.csv', sep=';')
    df_curricular = pd.read_csv('componente_curricular.csv', sep=';')
    df = df_lotacao.merge(df_curricular, on='codigo')
    make_post(df, api_url+'componente')

if __name__ == '__main__':
    insert('unidade.csv', api_url+'unidade')
    insert('docente.csv', api_url+'docente')
    insert_componentes()
    insert('turma.csv', api_url+'turma')
    insert('avaliacao.csv', api_url+'avaliacao')

{"timestamp":"2018-06-14T20:08:44.763+0000","status":500,"error":"Internal Server Error","message":"java.sql.SQLException: Parameter index out of range (3 > number of parameters, which is 2).","path":"/unidade"}
{"nome":"ADJA FERREIRA DE ANDRADE","formacao":"DOUTORADO","data_admissao":"2012-11-01","docente":5755281,"id_unidade":{"lotacao":"INSTITUTO METROPOLE DIGITAL","unidade":6069}}
{"nome":"ANDERSON PAIVA CRUZ","formacao":"DOUTORADO","data_admissao":"2013-04-25","docente":5755819,"id_unidade":{"lotacao":"INSTITUTO METROPOLE DIGITAL","unidade":6069}}
{"nome":"ANDRE LUIZ DA SILVA SOLINO","formacao":"MESTRADO","data_admissao":"2016-01-19","docente":5757969,"id_unidade":{"lotacao":"INSTITUTO METROPOLE DIGITAL","unidade":6069}}
{"nome":"ANNA GISELLE CAMARA DANTAS RIBEIRO RODRIGUES","formacao":"DOUTORADO","data_admissao":"2017-05-26","docente":5759298,"id_unidade":{"lotacao":"INSTITUTO METROPOLE DIGITAL","unidade":6069}}
{"nome":"APUENA VIEIRA GOMES","formacao":"DOUTORADO","data_admissao"

{"nome":"REGIVAN HUGO NUNES SANTIAGO","formacao":"DOUTORADO","data_admissao":"2002-02-19","docente":24798,"id_unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","unidade":125}}
{"nome":"ROBERTA DE SOUZA COELHO","formacao":"DOUTORADO","data_admissao":"2009-07-01","docente":5752649,"id_unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","unidade":125}}
{"nome":"SELAN RODRIGUES DOS SANTOS","formacao":"DOUTORADO","data_admissao":"2006-01-09","docente":62353,"id_unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","unidade":125}}
{"nome":"SILVIA MARIA DINIZ MONTEIRO MAIA","formacao":"DOUTORADO","data_admissao":"2014-02-21","docente":5756412,"id_unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","unidade":125}}
{"nome":"THAIS VASCONCELOS BATISTA","formacao":"DOUTORADO","data_admissao":"1996-12-02","docente":23809,"id_unidade":{"lotacao":"DEPARTAMENTO DE INFORMATICA E MATEMATICA APLICADA","unidade":125}}
{"nome":

{"codigo":"DIM0308","componente":21756,"nome":"QUALIDADE EM SOFTAWARE","id_unidade":null}
{"codigo":"DIM0349","componente":21757,"nome":"ENGENHARIA DE REQUISITOS E DESIGN DE SOFTWARE","id_unidade":null}
{"codigo":"DIM0352","componente":21758,"nome":"INTERACAO HUMANO-COMPUTADOR","id_unidade":null}
{"codigo":"DIM0439","componente":21759,"nome":"TEORIA DA COMPUTACAO","id_unidade":null}
{"codigo":"DIM0330","componente":21760,"nome":"LINGUAGENS FORMAIS","id_unidade":null}
{"codigo":"DIM0328","componente":21761,"nome":"ALGORITMOS E ESTRUTURAS DE DADOS II","id_unidade":null}
{"codigo":"DIM0342","componente":21762,"nome":"ALGORITMOS E ESTRUTURAS DE DADOS III","id_unidade":null}
{"codigo":"DIM0357","componente":21763,"nome":"BANCO DE DADOS","id_unidade":null}
{"codigo":"DIM0410","componente":21764,"nome":"TREINAMENTO PARA COMPETICOES DE PROGRAMACAO","id_unidade":null}
{"codigo":"DIM0337","componente":21765,"nome":"COMPUTABILIDADE","id_unidade":null}
{"codigo":"DIM0331","componente":21766,"nome"

{"codigo":"DIM0098","componente":26923,"nome":"TOPICOS ESPECIAIS EM COMPUTACAO IX","id_unidade":null}
{"codigo":"DIM0099","componente":26924,"nome":"TOPICOS ESPECIAIS EM COMPUTACAO X","id_unidade":null}
{"codigo":"DIM0100","componente":26925,"nome":"ANALISES DE SINAIS E SISTEMA","id_unidade":null}
{"codigo":"DIM0101","componente":26926,"nome":"CIRCUITOS LOGICOS","id_unidade":null}
{"codigo":"DIM0130","componente":26927,"nome":"INTRODUCAO A INFORMATICA","id_unidade":null}
{"codigo":"DIM0311","componente":26928,"nome":"TOPICOS ESPECIAIS EM ENGENHARIA DE COMPUTACAO I","id_unidade":null}
{"codigo":"DIM0312","componente":26929,"nome":"TOPICOS ESPECIAIS EM ENGENHARIA DE COMPUTACAO II","id_unidade":null}
{"codigo":"DIM0333","componente":26930,"nome":"ELEM.DE PESQ.OPER.APLIC. A INDUST.DO PETROLEO","id_unidade":null}
{"codigo":"DIM0340","componente":26931,"nome":"FORMACAO HUMANISTICA EM COMPUTACAO","id_unidade":null}
{"codigo":"DIM0351","componente":26932,"nome":"PLANEJAMENTO E CONTROLE DE PROJ

{"codigo":"DIM0491","componente":2040701,"nome":"TÓPICOS ESPECIAIS EM COMPUTAÇÃO XII","id_unidade":null}
{"codigo":"DIM0492","componente":2040702,"nome":"TÓPICOS ESPECIAIS EM COMPUTAÇÃO XIII","id_unidade":null}
{"codigo":"DIM0493","componente":2040703,"nome":"TÓPICOS ESPECIAIS EM COMPUTAÇÃO XVI","id_unidade":null}
{"codigo":"DIM0494","componente":2040704,"nome":"TÓPICOS ESPECIAIS EM COMPUTAÇÃO XV","id_unidade":null}
{"codigo":"DIM0495","componente":2040705,"nome":"TÓPICOS ESPECIAIS EM COMPUTAÇÃO XIV","id_unidade":null}
{"codigo":"DIM0448","componente":2040707,"nome":"QUALIDADE DE SOFTWARE","id_unidade":null}
{"codigo":"DIM0451","componente":2040950,"nome":"COMPUTAÇÃO GRÁFICA I","id_unidade":null}
{"codigo":"DIM0450","componente":2040951,"nome":"FUNDAMENTOS MATEMÁTICOS DA COMPUTAÇÃO","id_unidade":null}
{"codigo":"DIM0481","componente":2041250,"nome":"PROGRAMAÇÃO ORIENTADA A OBJETOS","id_unidade":null}
{"codigo":"DIM0501","componente":2054904,"nome":"BOAS PRÁTICAS DE PROGRAMAÇÃO","id_uni

...

{"id":3691,"aprovados":0.8103448276,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5758730,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57597076,"id_componente":null},"qtd_discentes":51,"postura_profissional":9.86,"postura_profissional_DP":0.35,"atuacao_profissional":9.73,"atuacao_profissional_DP":0.69,"media_aprovados":6.9361702128}
{"id":3692,"aprovados":0.6551724138,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5758730,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57597094,"id_componente":null},"qtd_discentes":51,"postura_profissional":9.9,"postura_profissional_DP":0.3,"atuacao_profissional":9.54,"atuacao_profissional_DP":0.85,"media_aprovados":6.5078947368}
{"id":3693,"aprovados":0.6037735849,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5758730,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57600128,"id_comp

{"id":3718,"aprovados":0.75,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5756342,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57576782,"id_componente":null},"qtd_discentes":9,"postura_profissional":10.0,"postura_profissional_DP":0.0,"atuacao_profissional":9.94,"atuacao_profissional_DP":0.35,"media_aprovados":7.8875}
{"id":3719,"aprovados":0.7894736842,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5756342,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57587403,"id_componente":null},"qtd_discentes":16,"postura_profissional":9.92,"postura_profissional_DP":0.39,"atuacao_profissional":9.97,"atuacao_profissional_DP":0.17,"media_aprovados":7.2533333333}
{"id":3720,"aprovados":0.5294117647,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5756342,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57587434,"id_componente":null},

{"id":3744,"aprovados":0.6268656716,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5757905,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57576726,"id_componente":null},"qtd_discentes":34,"postura_profissional":9.78,"postura_profissional_DP":0.55,"atuacao_profissional":9.34,"atuacao_profissional_DP":0.97,"media_aprovados":6.8571428571}
{"id":3745,"aprovados":0.7647058824,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5757905,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57576747,"id_componente":null},"qtd_discentes":34,"postura_profissional":9.63,"postura_profissional_DP":1.29,"atuacao_profissional":8.96,"atuacao_profissional_DP":2.08,"media_aprovados":7.6307692308}
{"id":3746,"aprovados":0.4285714286,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5757905,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57586373,"id_co

{"id":3765,"aprovados":0.4411764706,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5757620,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57564865,"id_componente":null},"qtd_discentes":44,"postura_profissional":9.82,"postura_profissional_DP":0.45,"atuacao_profissional":9.47,"atuacao_profissional_DP":1.22,"media_aprovados":6.536}
{"id":3766,"aprovados":0.3707865169,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5757620,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57576529,"id_componente":null},"qtd_discentes":48,"postura_profissional":9.61,"postura_profissional_DP":0.74,"atuacao_profissional":9.27,"atuacao_profissional_DP":1.3,"media_aprovados":6.7818181818}
{"id":3767,"aprovados":0.72,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5757620,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57576780,"id_componente":null},

{"id":3788,"aprovados":0.1111111111,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5757958,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57597928,"id_componente":null},"qtd_discentes":43,"postura_profissional":9.78,"postura_profissional_DP":0.65,"atuacao_profissional":8.42,"atuacao_profissional_DP":2.46,"media_aprovados":6.25}
{"id":3789,"aprovados":0.6428571429,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5757958,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57600757,"id_componente":null},"qtd_discentes":9,"postura_profissional":8.5,"postura_profissional_DP":2.12,"atuacao_profissional":8.38,"atuacao_profissional_DP":1.41,"media_aprovados":7.5833333333}
{"id":3790,"aprovados":0.4311377246,"id_docente":{"nome":null,"formacao":null,"data_admissao":null,"docente":5757969,"id_unidade":null},"id_turma":{"ano":null,"nivel":null,"periodo":null,"turma":57576530,"id_componente":